In [1]:
# Carregar llibreries

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import numpy as np
import random
import time
from google.colab import files

In [3]:
# Carrega el fitxer Excel

# INFORME GPG: Inventari de drets immaterials - Dates de vigència

# nom_fitxer = '/content/Inventaridedretsimmaterials-Datesdevigència (2).xlsx'  # Canvia a la ruta correcta del teu fitxer
# df = pd.read_excel(nom_fitxer, header=13)  # Ajusta el valor de 'header' segons la posició de les capçaleres

# Carrega l'arxiu XLSX
uploaded = files.upload()

# Obtenir el nom de l'arxiu carregat
nom_arxiu = list(uploaded.keys())[0]

# Llegir l'arxiu XLSX amb pandas
df = pd.read_excel(nom_arxiu, sheet_name="oepm", header=13)  # Ajusta el valor de 'header' segons la posició de les capçaleres

# Selecciona les columnes que necessites
columnes_seleccionades = [
    "Codi registre",
    "Data fi vigència",
    "Data inici vigència",
]

df = df[columnes_seleccionades]


Saving ACT-marques-dates-vigència.xlsx to ACT-marques-dates-vigència.xlsx


In [4]:
df

,Codi registre,Data fi vigència,Data inici vigència
0,2675467,2025-10-20,2005-10-20
1,2753142,2026-10-11,1996-10-11
2,2753255,2027-02-02,2007-02-02
3,2753260,2027-02-02,2007-02-02
4,2753757,2027-02-06,2007-02-06
5,2753765,2027-02-06,2007-02-06
6,2768443,2027-04-24,2007-04-24
7,2855924,2028-12-10,2008-12-10
8,4162453,2032-03-15,2022-03-15
9,4162462,2032-03-15,2022-03-15


In [6]:
# Filtrar XLSX

# Obtenir la data d'avui
avui = datetime.now()

df2 = df
# print(df2["Codi registre"])

In [7]:
def separar(cadena):
    # Patró de cerca per la data (dd/mm/yyyy)
    patro_data = re.compile(r'\b(\d{1,2}/\d{1,2}/\d{4})\b')

    # Cerca la data a la cadena
    coincidencia_data = patro_data.search(cadena)

    if coincidencia_data:
        # Si es troba la data, separa el text i la data
        text = cadena[:coincidencia_data.start()].strip()
        data = coincidencia_data.group()
    else:
        # Si no es troba la data, retorna la cadena completa com a text i deixa la data buida
        text = cadena
        data = ""

    return text, data

# Exemple d'ús
# cadena = "PUBL.CONCES.RENOV. DE F.RESOL. 06/11/2023"
# text, data = separar(cadena)


In [16]:
def dades(codi):

    url = "http://consultas2.oepm.es/ceo/jsp/busqueda/consultaExterna.xhtml?numExp=M" + codi

    dades = {}  # Crear un diccionari per emmagatzemar les dades

    dades['Codi'] = codi

    try:
        response = requests.get(url)

        if response.status_code == 200:

          try:

            # Parseja el contingut HTML amb BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Troba l'element <h4> amb el text "Actos de tramitación:"
            h4_actos = soup.find('h4', string=lambda text: "Actos de tramitación:" in text if text else False)
            h4_actos = h4_actos.find_next('div', class_='datos')

            # Troba tots els elements <td> amb l'atribut role="gridcell"
            td_elements = h4_actos.find_all('td', {'role': 'gridcell'})

            # Verifica si hi ha algun element
            if td_elements:
                # Obtenir l'últim element i extreure'n el text
                dades['last_td'] = td_elements[-1].get_text(strip=True)
                dades['Data publicació'] = td_elements[-2].get_text(strip=True)

                dades['Anotació'], dades['Data anotació'] = separar(dades['last_td'])

            # Busca l'element <h4> que contingui el text "Fecha presentación solicitud otorgada:"
            presentacio_h4 = soup.find('h4', string=lambda text: "Fecha presentación solicitud otorgada:" in text if text else False)

            # Obte el text de l'element següent (en aquest cas, l'element <p>)
            if presentacio_h4:
                dades['Data de presentació'] = presentacio_h4.find_next('p').get_text(strip=True)

            # Busca l'element <h4> que contingui el text "Estado:"
            estado_h4 = soup.find('h4', string=lambda text: "Estado:" in text if text else False)

            # Obte el text de l'element següent (en aquest cas, l'element <p>)
            if estado_h4:
                dades['Estat'] = estado_h4.find_next('p').get_text(strip=True)

            # Troba l'element <div> amb l'id "marco-imagen-sitamar"
            div_marco_imagen = soup.find('div', {'id': 'marco-imagen-sitamar'})

            if div_marco_imagen:
                # Troba l'element <img> dins del div
                img_element = div_marco_imagen.find('img')

                if img_element:
                    # Obté el valor de l'atribut src
                    src_valor = img_element.get('src', '')

                    # Imprimeix el resultat
                    dades['Imatge'] = "http://consultas2.oepm.es"+src_valor
                else:
                    dades['Imatge'] = ""

            # Troba l'element <h3>
            h3_element = soup.find('h3')

            if h3_element:
                # Obté el text de l'element <h3>
                text_h3 = h3_element.get_text(strip=True)

                # Utilitza una expressió regular per seleccionar el segon grup
                match = re.match(r'(.*)\s-\s(.*)', text_h3)

                if match:
                    segon_grup = match.group(2)
                    dades['Denominació'] = segon_grup

            # Troba l'element <h4>
            h4_element = soup.find('h4', string=lambda text: "Tipo:" in text if text else False)

            if h4_element:
                # Obtenir el text de l'element <h4>
                text_h4 = h4_element.find_next('p').get_text(strip=True)

                dades['Tipus'] = text_h4

            # Extreu el text que segueix a <span class="campo">Nombre:</span>
            nom_span = soup.find('span', class_='campo', string="Nombre:")
            if nom_span:
                text_nom = nom_span.find_next('div', class_='datos-sitamar-divSubRight').text.strip()
                # Comprova si hi ha l'etiqueta <br/>
                if "\n" in text_nom:
                    # Divideix el text en dos blocs amb l'etiqueta <br />
                    motiu, titular = map(str.strip, text_nom.split('\n', 1))
                    dades['Titular OEPM'] = titular
                    dades['Motiu'] = motiu
                else:
                    # Si no hi ha <br />, posa tot el text a 'titular'
                    dades['Titular OEPM'] = text_nom
                    dades['Motiu'] = ""

            dades['Url'] = url
          except:
              dades['Estat'] = "Núm. registre incorrecte"

        else:
            print(f"S'ha produït un error en la sol·licitud. Codi d'estat: {response.status_code}")

    except requests.RequestException as e:
        print(f"Error de connexió: {e}")

    return dades  # Retorna el diccionari amb les dades

In [9]:
'''
# Exemple d'ús

resultats = dades("2551345")

# Ara pots fer el que vulguis amb les dades obtingudes
if resultats:
    # Fer quelcom amb les dades, com ara imprimir-les
    print("Dades obtingudes:")
    print(f"codi: {resultats.get('Codi')}")
    print(f"denominació: {resultats.get('Denominació')}")
    print(f"estado_text: {resultats.get('Estat')}")
    print(f"data_publicacio: {resultats.get('Data publicació')}")
    print(f"anotacio: {resultats.get('Anotació')}")
    print(f"anotacio_data: {resultats.get('Data anotació')}")
    print(f"imatge: {resultats.get('Imatge')}")
    print(f"titular: {resultats.get('Titular OEPM')}")
    print(f"motiu: {resultats.get('Motiu')}")
    print(f"tipus: {resultats.get('Tipus')}")
    print(f"Url: {resultats.get('Url')}")
'''

'\n# Exemple d\'ús\n\nresultats = dades("2551345")\n\n# Ara pots fer el que vulguis amb les dades obtingudes\nif resultats:\n    # Fer quelcom amb les dades, com ara imprimir-les\n    print("Dades obtingudes:")\n    print(f"codi: {resultats.get(\'Codi\')}")\n    print(f"denominació: {resultats.get(\'Denominació\')}")\n    print(f"estado_text: {resultats.get(\'Estat\')}")\n    print(f"data_publicacio: {resultats.get(\'Data publicació\')}")\n    print(f"anotacio: {resultats.get(\'Anotació\')}")\n    print(f"anotacio_data: {resultats.get(\'Data anotació\')}")\n    print(f"imatge: {resultats.get(\'Imatge\')}")\n    print(f"titular: {resultats.get(\'Titular OEPM\')}")\n    print(f"motiu: {resultats.get(\'Motiu\')}")\n    print(f"tipus: {resultats.get(\'Tipus\')}")\n    print(f"Url: {resultats.get(\'Url\')}")\n'

In [17]:
# Baixar la informació

marques_revisades = pd.DataFrame()

id = df2["Codi registre"]

print("Marques a processar: ", len(id), "\n")

for index, x in enumerate(id):
    print(str(round(index/len(id)*100, 2))+"%")
    resultats = dades(str(x))
    marques_revisades = pd.concat([marques_revisades, pd.DataFrame([resultats])], ignore_index=True)
    time.sleep(random.uniform(10, 15))
print("100.00 %")

marques_revisades.drop('last_td', axis=1, inplace=True)
marques_revisades

Marques a processar:  11 

0.0%
9.09%
18.18%
27.27%
36.36%
45.45%
54.55%
63.64%
72.73%
81.82%
90.91%
100.00 %


,Codi,Data publicació,Anotació,Data anotació,Data de presentació,Estat,Imatge,Denominació,Tipus,Titular OEPM,Motiu,Url
0,2675467,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,20/10/2005,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,CENTRES BTT CATALUNYA,Mixta,AGÈNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
1,2753142,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,11/10/1996,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,TURISME DE CATALUNYA,Mixta,AGÉNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
2,2753255,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,02/02/2007,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,DESTINACIO DE TURISME ESPORTIU,Mixta,AGÈNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
3,2753260,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,02/02/2007,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,DESTINO DE TURISMO DEPORTIVO,Mixta,AGÈNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
4,2753757,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,06/02/2007,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,DESTINACIO DE TURISME FAMILIAR,Mixta,AGÈNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
5,2753765,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,06/02/2007,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,DESTINO DE TURISMO FAMILIAR,Mixta,AGÈNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
6,2768443,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0796 (POD...",,24/04/2007,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,CAT RUTES DE CATALUNYA,Mixta,AGÈNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2010.03250 DE F. RESOL....,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
7,2855924,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,10/12/2008,En vigor,,TRANSCATALUNYA,Denominativa estándar,AGÉNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
8,4162453,23/12/2022,PUBLIC. CONCESION DE F. RESOL.,16/12/2022,15/03/2022,En vigor,,L'ESTALVIATGE,Denominativa,AGÈNCIA CATALANA DE TURISME,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
9,4162462,07/11/2022,PUBLIC. CONCESION DE F. RESOL.,28/10/2022,15/03/2022,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,L'ESTALVIATGE,Figurativa,AGÈNCIA CATALANA DE TURISME,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...


In [18]:
import pandas as pd

# Defineix l'ordre desitjat de les columnes
ordre_columnes = ['Codi', 'Denominació', 'Tipus', 'Estat', 'Data de presentació', 'Data publicació', 'Anotació', 'Data anotació', 'Titular OEPM', 'Motiu', 'Imatge', 'Url']

# Crea un DataFrame amb les columnes desitjades
marques_revisades_2 = pd.DataFrame(columns=ordre_columnes)

# Afegeix les dades existents al teu DataFrame original
marques_revisades_2 = marques_revisades_2.append(marques_revisades, ignore_index=True)

# Reordena les columnes del DataFrame
marques_revisades = marques_revisades_2[ordre_columnes]
marques_revisades = marques_revisades.fillna("")

marques_revisades

<ipython-input-18-79a6356f4809>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  marques_revisades_2 = marques_revisades_2.append(marques_revisades, ignore_index=True)


,Codi,Denominació,Tipus,Estat,Data de presentació,Data publicació,Anotació,Data anotació,Titular OEPM,Motiu,Imatge,Url
0,2675467,CENTRES BTT CATALUNYA,Mixta,En vigor,20/10/2005,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,AGÈNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
1,2753142,TURISME DE CATALUNYA,Mixta,En vigor,11/10/1996,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,AGÉNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
2,2753255,DESTINACIO DE TURISME ESPORTIU,Mixta,En vigor,02/02/2007,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,AGÈNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
3,2753260,DESTINO DE TURISMO DEPORTIVO,Mixta,En vigor,02/02/2007,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,AGÈNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
4,2753757,DESTINACIO DE TURISME FAMILIAR,Mixta,En vigor,06/02/2007,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,AGÈNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
5,2753765,DESTINO DE TURISMO FAMILIAR,Mixta,En vigor,06/02/2007,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,AGÈNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
6,2768443,CAT RUTES DE CATALUNYA,Mixta,En vigor,24/04/2007,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0796 (POD...",,AGÈNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2010.03250 DE F. RESOL....,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
7,2855924,TRANSCATALUNYA,Denominativa estándar,En vigor,10/12/2008,14/10/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0535 (POD...",,AGÉNCIA CATALANA DE TURISME,POR TRANSFERENCIA NUM. 2011.00019 DE F. RESOL....,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
8,4162453,L'ESTALVIATGE,Denominativa,En vigor,15/03/2022,23/12/2022,PUBLIC. CONCESION DE F. RESOL.,16/12/2022,AGÈNCIA CATALANA DE TURISME,,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
9,4162462,L'ESTALVIATGE,Figurativa,En vigor,15/03/2022,07/11/2022,PUBLIC. CONCESION DE F. RESOL.,28/10/2022,AGÈNCIA CATALANA DE TURISME,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...


In [21]:
marques_revisades.to_excel(f'marques_oepm_extracció_{avui.year}{avui.month:02d}{avui.day:02d}.xlsx', index=False)

In [22]:
files.download(f'marques_oepm_extracció_{avui.year}{avui.month:02d}{avui.day:02d}.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>